In [2]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [111]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

[autoreload of mcr_py failed: Traceback (most recent call last):
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/mcr_py/__init__.py", line 3, in <module>
    __doc__ = mcr_py.__doc__
NameError: name 'mcr_py' is not defined
]


In [112]:
from mcr_py import run_mlc_with_bags, GraphCache

In [113]:

from typing import Any, Tuple

import pandas as pd
import geopandas as gpd

import mcr_py
from mcr_py import GraphCache
import pyrosm
from package import storage
from package.logger import Timed
from package.mcr.path import PathManager
from package.osm import osm, graph
from package.rust.bag import convert_to_intermediate_bags


ACCURACY = 4
ACCURACY_MULTIPLIER = 10 ** (ACCURACY - 1)

AVG_WALKING_SPEED = 1.4  # m/s
AVG_BIKING_SPEED = 4.0  # m/s



In [114]:
def get_reverse_map(d: dict[Any, Any]) -> dict[Any, Any]:
    return {v: k for k, v in d.items()}
def prefix_id(
    gdf: pd.DataFrame, prefix: str, column: str, save_old=False
) -> pd.DataFrame:
    if save_old:
        gdf[f"{column}_old"] = gdf[column]
    gdf[column] = prefix + gdf[column].astype(str)

    return gdf


def get_graph(
    osm_reader: pyrosm.OSM, stops_df: gpd.GeoDataFrame
) -> Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    with Timed.info("Getting OSM graph"):
        nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, stops_df)

    return nodes, edges


def mark_bicycles(nodes: pd.DataFrame) -> pd.DataFrame:
    nodes["has_bicycle"] = False
    nodes.loc[nodes.sample(100).index, "has_bicycle"] = True
    return nodes


def create_multi_modal_graph(
    nodes: pd.DataFrame, edges: pd.DataFrame
) -> dict[str, pd.DataFrame]:
    walking_nodes = nodes.copy()
    bike_nodes = nodes.copy()
    walking_edges = edges.copy()
    bike_edges = edges.copy()

    walking_nodes = prefix_id(walking_nodes, "W", "id", save_old=True)
    bike_nodes = prefix_id(bike_nodes, "B", "id", save_old=True)

    walking_edges = prefix_id(walking_edges, "W", "u")
    walking_edges = prefix_id(walking_edges, "W", "v")
    bike_edges = prefix_id(bike_edges, "B", "u")
    bike_edges = prefix_id(bike_edges, "B", "v")

    transfer_edges = create_transfer_edges(nodes)

    walking_edges = add_travel_time(walking_edges, AVG_WALKING_SPEED)
    bike_edges = add_travel_time(bike_edges, AVG_BIKING_SPEED)

    bike_edges["travel_time_bike"] = bike_edges["travel_time"]

    edges = combine_edges(walking_edges, bike_edges, transfer_edges)
    nodes = pd.concat([walking_nodes, bike_nodes])
    return {
        "nodes": nodes,
        "edges": edges,
        "walking_nodes": walking_nodes,
        "walking_edges": walking_edges,
    }


# create transfer edges from bike to walk at all nodes
def create_transfer_edges(nodes: pd.DataFrame):
    transfer_edges_values: pd.Series = nodes.apply(
        lambda x: ["B" + str(x.id), "W" + str(x.id), 0], axis=1
    )  # type: ignore
    transfer_edges = pd.DataFrame(
        transfer_edges_values.tolist(), columns=["u", "v", "length"]
    )

    return transfer_edges


def add_travel_time(edges: pd.DataFrame, speed: float) -> pd.DataFrame:
    edges["travel_time"] = edges.length / speed

    return edges


def combine_edges(
    walking_edges: pd.DataFrame,
    bike_edges: pd.DataFrame,
    transfer_edges: pd.DataFrame,
) -> pd.DataFrame:
    edges = pd.concat([walking_edges, bike_edges, transfer_edges], ignore_index=True)

    # fill travel_time for transfer edges and
    # travel_time_bike for walking and transfer edges
    edges = edges.fillna(0)

    return edges


def add_multi_modal_weights(edges: pd.DataFrame) -> pd.DataFrame:
    edges["weights"] = (
        "("
        + (edges["travel_time"].round(ACCURACY) * ACCURACY_MULTIPLIER)
        .astype(int)
        .astype(str)
        + ",0)"
    )
    edges["hidden_weights"] = (
        "("
        + (edges["travel_time_bike"].round(ACCURACY) * ACCURACY_MULTIPLIER)
        .astype(int)
        .astype(str)
        + ")"
    )

    return edges


def add_single_modal_weights(edges: pd.DataFrame) -> pd.DataFrame:
    edges["weights"] = (
        "("
        + (edges["travel_time"].round(ACCURACY) * ACCURACY_MULTIPLIER)
        .astype(int)
        .astype(str)
        + ")"
    )
    return edges


def reset_node_ids(
    nodes: pd.DataFrame, edges: pd.DataFrame
) -> Tuple[pd.DataFrame, pd.DataFrame, dict[Any, int]]:
    node_map = {}
    for i, node_id in enumerate(nodes.id.unique()):
        node_map[node_id] = i

    nodes["old_id"] = nodes["id"]
    nodes["id"] = nodes["id"].map(node_map)
    edges["u"] = edges["u"].map(node_map)
    edges["v"] = edges["v"].map(node_map)

    total_na = edges.isna().sum().sum() + nodes.isna().sum().sum()
    if total_na > 0:
        raise ValueError(f"Found {total_na} NaNs in graph")

    return nodes, edges, node_map


In [115]:
stops_path = "../data/cleaned/stops.csv"
city_id = "Koeln"
osm_path = ""

with Timed.info("Reading stops"):
	stops_df = storage.read_gdf(stops_path)

with Timed.info("Preparing graphs"):
	osm_reader = osm.get_osm_reader_for_city_id_or_osm_path(city_id, osm_path)
	nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, stops_df)
	nxgraph = graph.create_nx_graph(osm_reader, nodes, edges)

	nodes: pd.DataFrame = nodes[["id"]]  # type: ignore
	edges: pd.DataFrame = edges[["u", "v", "length"]]  # type: ignore

	stops_df = graph.add_nearest_node_to_stops(stops_df, nxgraph)

	nodes = mark_bicycles(nodes)

	graph_components = create_multi_modal_graph(nodes, edges)
	nodes, edges, walking_nodes, walking_edges = (
		graph_components["nodes"],
		graph_components["edges"],
		graph_components["walking_nodes"],
		graph_components["walking_edges"],
	)

	nodes, edges, node_map = reset_node_ids(nodes, edges)
	walking_nodes, walking_edges, walking_node_map = reset_node_ids(
		walking_nodes, walking_edges
	)
	reverse_node_map = get_reverse_map(node_map)
	reverse_walking_node_map = get_reverse_map(walking_node_map)

	edges = add_multi_modal_weights(edges)
	walking_edges = add_single_modal_weights(walking_edges)

	raw_edges = edges[["u", "v", "weights"]].to_dict("records")
	raw_walking_edges = walking_edges[["u", "v", "weights"]].to_dict("records")

bicycle_transfer_nodes_walking_node_ids = walking_nodes[
	walking_nodes["has_bicycle"]
].id.values

with Timed.info("Creating graph cache"):
	gc = GraphCache()
	gc.set_graph(raw_edges)
	walking_gc = GraphCache()
	walking_gc.set_graph(raw_walking_edges)

with Timed.info("Running Dijkstra step"):
	bags = mcr_py.run_mlc(walking_gc, 0)  # type: ignore

path_manager = PathManager()
intermediate_bags = convert_to_intermediate_bags(bags)
path_manager.extract_all_paths_from_bags(intermediate_bags)

# translates a node id from the walking graph to the corresponding bicycle
# node id from the multi-modal graph
def translate_walking_node_id_to_bicycle_node_id(
	walking_node_id: int,
) -> int:
	original_walking_node = reverse_walking_node_map[walking_node_id]
	original_bicycle_node = original_walking_node.replace("W", "B")
	bicycle_node_id = node_map[original_bicycle_node]
	return bicycle_node_id

# filter bags at bicycle nodes
bicycle_bags = {
	translate_walking_node_id_to_bicycle_node_id(node_id): bag
	for node_id, bag in intermediate_bags.items()
	if node_id in bicycle_transfer_nodes_walking_node_ids
}


[10:47:18] INFO     Reading stops                                                                     ]8;id=59713;file:///tmp/ipykernel_8359/25483325.py\25483325.py]8;;\:]8;id=785436;file:///tmp/ipykernel_8359/25483325.py#5\5]8;;\

           INFO     Reading stops done (0.04 seconds)                                                 ]8;id=747502;file:///tmp/ipykernel_8359/25483325.py\25483325.py]8;;\:]8;id=310375;file:///tmp/ipykernel_8359/25483325.py#5\5]8;;\

           INFO     Preparing graphs                                                                  ]8;id=830901;file:///tmp/ipykernel_8359/25483325.py\25483325.py]8;;\:]8;id=733875;file:///tmp/ipykernel_8359/25483325.py#8\8]8;;\

           INFO     Using existing OSM data                                                           ]8;id=777324;file:///tmp/ipykernel_8359/25483325.py\25483325.py]8;;\:]8;id=434299;file:///tmp/ipykernel_8359/25483325.py#8\8]8;;\

           INFO     Loading OSM network from cache                                                    ]8;id=904056;file:///tmp/ipykernel_8359/25483325.py\25483325.py]8;;\:]8;id=496685;file:///tmp/ipykernel_8359/25483325.py#8\8]8;;\

...

           INFO     queue size: 89                                                                       ]8;id=839717;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=929927;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 69                                                                       ]8;id=949159;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=18736;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     queue size: 19                                                                       ]8;id=705509;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=349487;file:///home/moritz/dev/uni/mlc/src/mlc.rs#215\215]8;;\

           INFO     Done! queue size: 0                                                                  ]8;id=205272;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=91073;file:///home/moritz/dev/uni/mlc/src/mlc.rs#226\226]8;;\

           INFO     Running Dijkstra step done (0.71 seconds)                                        ]8;id=749609;file:///tmp/ipykernel_8359/25483325.py\25483325.py]8;;\:]8;id=634613;file:///tmp/ipykernel_8359/25483325.py#51\51]8;;\

In [158]:
data = storage.read_any_dict("/home/moritz/dev/uni/mcr-py/data/bags.pkl")
bags = data["intermediate_bags"]
new_bags = data["new_intermediate_bags"]
path_manager = data["path_manager"]
node_map = data["node_map"]
walking_node_map = data["walking_node_map"]
reverse_node_map = {v: k for k, v in node_map.items()}
reverse_walking_node_map = {v: k for k, v in walking_node_map.items()}

In [118]:
walking_node_map["W1992584223"]

KeyError: 'W1992584223'

In [120]:
all_labels = [label for bag in new_bags.values() for label in bag]
all_labels[0].values
all_values = [label.values for label in all_labels]
all_hidden_values = [label.hidden_values for label in all_labels]
df = pd.DataFrame(all_values, columns=["eat", "cost"])
hidden_df = pd.DataFrame(all_hidden_values, columns=["bicycle_duration"])
df["bicycle_duration"] = hidden_df["bicycle_duration"]
df = df / 60
df.max()

eat                 38.966667
cost                 0.000000
bicycle_duration    30.366667
dtype: float64

In [152]:
import os
import folium
from package.mcr.path import PathType

In [123]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, stops_df)

[10:49:45] INFO     Reading stops                                                                   ]8;id=841022;file:///tmp/ipykernel_8359/2041299943.py\2041299943.py]8;;\:]8;id=246481;file:///tmp/ipykernel_8359/2041299943.py#5\5]8;;\

           INFO     Reading stops done (0.05 seconds)                                               ]8;id=15752;file:///tmp/ipykernel_8359/2041299943.py\2041299943.py]8;;\:]8;id=961551;file:///tmp/ipykernel_8359/2041299943.py#5\5]8;;\

           INFO     Using existing OSM data                                                        ]8;id=37418;file:///tmp/ipykernel_8359/2041299943.py\2041299943.py]8;;\:]8;id=963984;file:///tmp/ipykernel_8359/2041299943.py#12\12]8;;\

           INFO     Getting OSM graph                                                              ]8;id=202016;file:///tmp/ipykernel_8359/2041299943.py\2041299943.py]8;;\:]8;id=574190;file:///tmp/ipykernel_8359/2041299943.py#16\16]8;;\

           INFO     Loading OSM network from cache                                                 ]8;id=531014;file:///tmp/ipykernel_8359/2041299943.py\2041299943.py]8;;\:]8;id=206219;file:///tmp/ipykernel_8359/2041299943.py#16\16]8;;\

           INFO     Getting OSM graph done (0.45 seconds)                                          ]8;id=190317;file:///tmp/ipykernel_8359/2041299943.py\2041299943.py]8;;\:]8;id=466679;file:///tmp/ipykernel_8359/2041299943.py#16\16]8;;\

In [189]:
translator_map = {
    PathType.WALKING: reverse_walking_node_map,
    PathType.CYCLING_WALKING: reverse_node_map,
}
no_prefix_reverse_walking_node_map = {
    k: int(v[1:]) for k, v in reverse_walking_node_map.items()
}
no_prefix_reverse_node_map = {k: int(v[1:]) for k, v in reverse_node_map.items()}
no_prefix_translator_map = {
    PathType.WALKING: no_prefix_reverse_walking_node_map,
    PathType.CYCLING_WALKING: no_prefix_reverse_node_map,
}


In [182]:

path = path_manager.reconstruct_and_translate_path_for_label(label, no_prefix_translator_map)
path

In [184]:
sample_label = list(bags.values())[0][0]
sample_node_id = int(reverse_walking_node_map[label.node_id].replace('W', ''))
nodes_by_id = nodes.set_index('id')
sample_node = nodes_by_id.loc[sample_node_id]

m = folium.Map(location=[sample_node.lat, sample_node.lon], zoom_start=15)
counter = 0
for bag in bags.values():
	if counter > 10:
		break
	for label in bag:
		counter += 1


		node_id = int(reverse_walking_node_map[label.node_id].replace('W', ''))
		node = nodes_by_id.loc[node_id]
		popup=f"node_id: {node_id}\n label: {round(label.values[0]/60, 2)}min"
		folium.CircleMarker(
			location=[node.lat, node.lon],
			radius=3,
			popup=popup,
			color='red',
		).add_to(m)

		# path
		paths = path_manager.reconstruct_and_translate_path_for_label(label, no_prefix_translator_map)
		for path in paths:
			path_nodes = [nodes_by_id.loc[node_id] for node_id in path.path]
			path_lat_lon = [(node.lat, node.lon) for node in path_nodes]
			folium.PolyLine(path_lat_lon, color='blue', weight=2).add_to(m)
m

In [194]:
color_map = {
	PathType.WALKING: 'red',
	PathType.CYCLING_WALKING: 'blue',
}

In [198]:
sample_label = list(new_bags.values())[0][0]
sample_node_id = int(reverse_node_map[label.node_id].replace('B', '').replace('W', ''))
nodes_by_id = nodes.set_index('id')
sample_node = nodes_by_id.loc[sample_node_id]

m = folium.Map(location=[sample_node.lat, sample_node.lon], zoom_start=15)
counter = 0
for bag in new_bags.values():
	if counter > 5:
		break
	for label in bag:
		counter += 1


		node_id = int(reverse_node_map[label.node_id].replace('B', '').replace('W', ''))
		node = nodes_by_id.loc[node_id]
		popup=f"node_id: {node_id}\n label: {round(label.values[0]/60, 2)}min"
		folium.CircleMarker(
			location=[node.lat, node.lon],
			radius=3,
			popup=popup,
			color='red',
		).add_to(m)

		# path
		paths = path_manager.reconstruct_and_translate_path_for_label(label, no_prefix_translator_map)
		for path in paths:
			path_nodes = [nodes_by_id.loc[node_id] for node_id in path.path]
			path_lat_lon = [(node.lat, node.lon) for node in path_nodes]
			color = color_map[path.path_type]
			folium.PolyLine(path_lat_lon, color=color, weight=2).add_to(m)
m